In [220]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
import numpy as np
from PIL import Image
import io
import os
import os.path
import glob
from datetime import datetime
from google.cloud import vision
from google.cloud.vision import types

In [281]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROKS-78e082836307.json"

plt.figure(figsize=(50, 40))
plt.style.use('dark_background')

images = glob.glob('data/info/original/*.jpg')
images.pop(images.index('data/info/original/Screenshot_20200727-203539_Rise of Kingdoms.jpg'))

<Figure size 3600x2880 with 0 Axes>

In [7]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [8]:
def change_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [9]:
def change_to_list(chars):
    tmp_texts = list()
    s = ""
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""
    
    return tmp_texts

In [197]:
start_time = datetime.now()

# crop & convert
for image in images:
    initial = Image.open(image)
    width = initial.size[0]
    height = initial.size[1]
    
    path, file_name = os.path.split(image)

    croped = initial.crop((width*0.2, height*0.1, width*0.80, height*0.98))

    im_blk_1 = Image.new('RGB', (303, 1267), (0, 0, 0))
    im_blk_2 = Image.new('RGB', (3000, 400), (0, 0, 0))
    im_blk_3 = Image.new('RGB', (3000, 250), (0, 0, 0))
    im_blk_4 = Image.new('RGB', (3000, 237), (0, 0, 0))

    x1, y1 = (0, 0)
    x2, y2 = (0, 150)
    x3, y3 = (0, 650)
    x4, y4 = (0, 1030)

    croped.paste(im_blk_1, (x1, y1, x1+303, y1+1267))
    croped.paste(im_blk_2, (x2, y2, x2+3000, y2+400))
    croped.paste(im_blk_3, (x3, y3, x3+3000, y3+250))
    croped.paste(im_blk_4, (x4, y4, x4+3000, y4+237))

    croped.save('data/info/croped/main/' + file_name)

    #kill
    croped = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))
    img_white = Image.new('RGB', (57, 231), (255, 255, 255, 255))
    
    croped.paste(img_white, (0, 0, 57, 231))
    croped.paste(img_white, (243, 0, 300, 231))

    croped.save('data/info/croped/kill/' + file_name)
    
    img_kill = cv2.imread('data/info/croped/kill/' + file_name)
    gray = cv2.cvtColor(img_kill, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        160,
        255,
        cv2.THRESH_BINARY_INV
    )
    cv2.imwrite('data/info/croped/kill/' + file_name, img_thresh)

end_time = datetime.now()

print (end_time - start_time)

0:00:25.373509


In [305]:
start_time = datetime.now()

# crop & convert
for image in images:
    initial = Image.open(image)
    width = initial.size[0]
    height = initial.size[1]
    
    path, file_name = os.path.split(image)

    crop_main = initial.crop((width*0.2, height*0.1, width*0.80, height*0.98))

    im_blk_1 = Image.new('RGB', (303, 1267), (0, 0, 0))
    im_blk_2 = Image.new('RGB', (3000, 400), (0, 0, 0))
    im_blk_3 = Image.new('RGB', (3000, 250), (0, 0, 0))
    im_blk_4 = Image.new('RGB', (3000, 237), (0, 0, 0))

    x1, y1 = (0, 0)
    x2, y2 = (0, 150)
    x3, y3 = (0, 650)
    x4, y4 = (0, 1030)

    crop_main.paste(im_blk_1, (x1, y1, x1+303, y1+1267))
    crop_main.paste(im_blk_2, (x2, y2, x2+3000, y2+400))
    crop_main.paste(im_blk_3, (x3, y3, x3+3000, y3+250))
    crop_main.paste(im_blk_4, (x4, y4, x4+3000, y4+237))

    #kill
    crop_kill = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))
    img_white = Image.new('RGB', (57, 231), (255, 255, 255, 255))
    
    crop_kill.paste(img_white, (0, 0, 57, 231))
    crop_kill.paste(img_white, (243, 0, 300, 231))

    mw, mh = crop_main.size
    kw, kh = crop_kill.size
    
    img_merge = Image.new('RGB', (mw, mh+kh), (0, 0, 0))

    img_merge.paste(crop_main, (0, 0, mw, mh))
    img_merge.paste(crop_kill, (0, mh, kw, mh+kh))
    
    img_merge.save('data/info/croped/' + file_name)
    
    #convert
    img = cv2.imread('data/info/croped/' + file_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        160,
        255,
        cv2.THRESH_BINARY_INV
    )
    
    cv2.imwrite('data/info/convert/' + file_name, img_thresh)

end_time = datetime.now()

print (end_time - start_time)

0:00:30.370981


In [306]:
start_time = datetime.now()
name = list()
power = list()
kill = list()
dead = list()
support = list()
t1 = list()
t2 = list()
t3 = list()
t4 = list()
t5 = list()

total = list()
total.append(name)
total.append(power)
total.append(kill)
total.append(dead)
total.append(support)
total.append(t1)
total.append(t2)
total.append(t3)
total.append(t4)
total.append(t5)

n = 1

for image in images:
    
    path, file_name = os.path.split(image)

    text = detect_text('data/info/convert/' + file_name)
    texts = change_to_list(text)

    for i in range(10):
        if i == 0:
            total[i].append(texts[0])
        else:
            total[i].append(change_to_number(texts[i]))
#     name.append(texts[0])
#     power.append(change_to_number(texts[1]))
#     kill.append(change_to_number(texts[2]))
#     dead.append(change_to_number(texts[3]))
#     support.append(change_to_number(texts[4]))
#     t1.append(change_to_number(texts[5]))
#     t2.append(change_to_number(texts[6]))
#     t3.append(change_to_number(texts[7]))
#     t4.append(change_to_number(texts[8]))
#     t5.append(change_to_number(texts[9]))
    print(n)
    n += 1

end_time = datetime.now()

print (end_time - start_time)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


IndexError: list index out of range

In [277]:
df = pd.DataFrame({'name': name,
                   'power':power,
                   'kill':kill,
                   'T1_kill':t1,
                   'T2_kill':t2,
                   'T3_kill':t3,
                   'T4_kill':t4,
                   'T5_kill':t5,
                   'dead':dead,
                   'support':support})

In [279]:
df.to_excel('data/info/result6.xlsx')

In [315]:
print(images[97])

data/info/original/Screenshot_20200727-204425_Rise of Kingdoms.jpg


In [319]:
initial = Image.open(images[131])
width = initial.size[0]
height = initial.size[1]

path, file_name = os.path.split(images[131])

crop_main = initial.crop((width*0.2, height*0.1, width*0.80, height*0.98))

im_blk_1 = Image.new('RGB', (303, 1267), (0, 0, 0))
im_blk_2 = Image.new('RGB', (3000, 400), (0, 0, 0))
im_blk_3 = Image.new('RGB', (3000, 250), (0, 0, 0))
im_blk_4 = Image.new('RGB', (3000, 237), (0, 0, 0))

x1, y1 = (0, 0)
x2, y2 = (0, 150)
x3, y3 = (0, 650)
x4, y4 = (0, 1030)

crop_main.paste(im_blk_1, (x1, y1, x1+303, y1+1267))
crop_main.paste(im_blk_2, (x2, y2, x2+3000, y2+400))
crop_main.paste(im_blk_3, (x3, y3, x3+3000, y3+250))
crop_main.paste(im_blk_4, (x4, y4, x4+3000, y4+237))

#kill
crop_kill = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))
img_white = Image.new('RGB', (57, 231), (255, 255, 255, 255))

crop_kill.paste(img_white, (0, 0, 57, 231))
crop_kill.paste(img_white, (243, 0, 300, 231))

mw, mh = crop_main.size
kw, kh = crop_kill.size

img_merge = Image.new('RGB', (mw, mh+kh), (0, 0, 0))

img_merge.paste(crop_main, (0, 0, mw, mh))
img_merge.paste(crop_kill, (0, mh, kw, mh+kh))

img_merge.save('data/info/croped/' + file_name)

#convert

img = cv2.imread('data/info/croped/' + file_name)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

ret, img_thresh = cv2.threshold(
    img_blurred, 
    150,
    255,
    cv2.THRESH_BINARY_INV
)

cv2.imwrite('data/info/convert/' + file_name, img_thresh)

True

In [321]:
path, file_name = os.path.split(images[97])
print(images[97])
text = detect_text('data/info/convert/' + file_name)
texts = change_to_list(text)
print(texts)
text

data/info/original/Screenshot_20200727-204425_Rise of Kingdoms.jpg
['전투력: 33,757,409', '처치: 11,236,444', '2,141,090', '1,981,018,186', '910,968', '1,037,491', '1,044,805', '6,373,648', '1,866,622']


'전투력: 33,757,409\n처치: 11,236,444\n2,141,090\n1,981,018,186\n910,968\n1,037,491\n1,044,805\n6,373,648\n1,866,622\n'